<a href="https://colab.research.google.com/github/sastrawanikadek/bangkit-assignment-5-face-recognition-CNN/blob/master/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import cv2
import os

def main():
    angka=1
    angka=str(angka)
    dim =(320,240)
    for file1 in os.listdir('gambar/'):
        for file in os.listdir('gambar/mukak ('+ angka +')/'):
            print('gambar/mukak ('+ angka +')/')
            img = cv2.imread('gambar/mukak ('+ angka +')/'+ file)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = cerah(img)
            resized = cv2.resize(img,dim, interpolation = cv2.INTER_AREA)
            cv2.imwrite('gambar/mukak ('+ angka +')/'+file, resized)
            print(file + " - DONE")
            print('Resized Dimensions : ',resized.shape)
        angka=int(angka)
        angka=angka+1
        angka=str(angka)
   
def cerah(img):
    tinggi = img.shape[0]
    lebar = img.shape[1]
    brightness = 50
    for i in np.arange(tinggi):
        for j in np.arange(lebar):
            a = img.item(i,j)
            b = a + brightness
            if b > 255:
                b = 255
            
            img.itemset((i,j), b)
    return img

if __name__ == "__main__":
    main()
    